In [4]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output, dash_table

# Step 1: Load Gyroscope CSV data (expects columns like time, x, y, z)
df = pd.read_csv("gyro_data.csv")

# Step 2: Create the Dash app layout
app = Dash(__name__)
app.layout = html.Div([
    html.H1("Gyroscope Data Dashboard"),

    dcc.Dropdown(
        id="chart-type",
        options=[
            {"label": "Line Chart", "value": "line"},
            {"label": "Scatter Plot", "value": "scatter"},
            {"label": "Histogram", "value": "hist"}
        ],
        value="line"
    ),
    dcc.Dropdown(
        id="variables",
        options=[
            {"label": "X-axis", "value": "x"},
            {"label": "Y-axis", "value": "y"},
            {"label": "Z-axis", "value": "z"}
        ],
        value=["x", "y", "z"],
        multi=True
    ),
    dcc.Input(id="num-samples", type="number", value=100, min=10, step=10),
    html.Div([
        html.Button("Previous", id="prev-btn", n_clicks=0),
        html.Button("Next", id="next-btn", n_clicks=0),
    ]),

    dcc.Graph(id="gyro-graph"),
    dash_table.DataTable(id="summary-table")
])

# Step 3: Add interactivity with a callback
@app.callback(
    [Output("gyro-graph", "figure"),
     Output("summary-table", "data"),
     Output("summary-table", "columns")],
    [Input("chart-type", "value"),
     Input("variables", "value"),
     Input("num-samples", "value"),
     Input("prev-btn", "n_clicks"),
     Input("next-btn", "n_clicks")]
)
def update_graph(chart_type, variables, num_samples, prev_clicks, next_clicks):
    start_index = max(0, (next_clicks - prev_clicks) * num_samples)
    end_index = start_index + num_samples
    dff = df.iloc[start_index:end_index]

    # Step 4: Generate the selected plot type
    if chart_type == "line":
        fig = px.line(dff, y=variables, title="Line Chart of Gyroscope Data")
    elif chart_type == "scatter":
        fig = px.scatter(dff, y=variables, title="Scatter Plot of Gyroscope Data")
    else:
        fig = px.histogram(dff, x=variables[0], title="Histogram of Selected Variable")

    # Step 5: Create a cleaner summary table (Minimum, Maximum, Average)
    summary = dff[variables].describe().loc[["min", "max", "mean"]]
    summary = summary.rename(index={"min": "Minimum", "max": "Maximum", "mean": "Average"})
    summary = summary.reset_index().rename(columns={"index": "Statistic"})

    data = summary.to_dict("records")
    columns = [{"name": i, "id": i} for i in summary.columns]

    return fig, data, columns

# Step 6: Run the app
if __name__ == "__main__":
    app.run(debug=True)
